In [27]:
!pip install gymnasium
!pip install swig
!pip install gymnasium[box2d]
from IPython import display
%matplotlib inline
import gymnasium as gym
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import time
import random

In [4]:
from PIL import Image
import numpy as np
def display_image(imageArray):
    display.display(Image.fromarray(np.array(imageArray),'RGB'),clear=True)

In [33]:
from collections import namedtuple
Transition  = namedtuple('Transition', ('state','action', 'state_next','reward'))

In [71]:
model = nn.Sequential()
model.add_module('fc1',nn.Linear(4,32))
model.add_module('relu1',nn.ReLU())
model.add_module('fc2', nn.Linear(32,32))
model.add_module('relu2', nn.ReLU())
model.add_module('fc2', nn.Linear(32,2))
a = []
a.append(Transition([1,2,3,4],[0],3,1))
a.append(Transition([5,5,7,6],[1],7,0))
a.append(Transition([6,2,3,4],[1],None,0))
batch = Transition(*zip(*a))

state_batch = torch.Tensor(batch.state)
action_batch = torch.Tensor(batch.action)
reward_batch = torch.Tensor(batch.reward)
non_final_next_states = torch.Tensor([s for s in batch.state_next if s is not None])
state_batch
model.eval()
model(state_batch).gather(1,action_batch.long())

tensor([[0.6006],
        [0.0000],
        [0.0000]], grad_fn=<GatherBackward0>)

In [46]:
NUM_DIZITIZED = 6
MAX_STEP = 500
NUM_EPISODE = 500
CAPACITY = 10000
memory = []
index = 0

model = nn.Sequential()
model.add_module('fc1',nn.Linear(4,32))
model.add_module('relu1',nn.ReLU())
model.add_module('fc2', nn.Linear(32,32))
model.add_module('relu2', nn.ReLU())
model.add_module('fc2', nn.Linear(32,2))

optimizer = optim.Adam(model.parameters(), lr=0.0001)

def push(state, action, state_next, reward):
    if len(memory) < CAPACITY:
        memory.append(None)
    memory[index] = Transition(state,action,state_next, reward)
    index = (index + 1) % CAPACITY

def sample(batch_size):
    return random.sample(memory,batch_size)
    
def reset(env):
      observation = env.reset()[0]
      s_a_history = [[observation, np.nan]]
      pi = [0.5,0.5]
      return observation, s_a_history, pi
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [
        np.digitize(cart_pos, bins=bins(-2.4, 2.4, NUM_DIZITIZED)),
        np.digitize(cart_v, bins=bins(-3.0, 3.0, NUM_DIZITIZED)),
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, NUM_DIZITIZED)),
        np.digitize(pole_v, bins=bins(-2.0, 2.0, NUM_DIZITIZED))
    ]
    return sum([x * (NUM_DIZITIZED**i) for i, x in enumerate(digitized)])

def get_action(s,Q,epsilon, pi=[0.5,0.5]):
    state = digitize_state(s)
    if np.random.rand() < epsilon:
        action = np.random.choice([0,1])
    else:
        action = np.argmax(Q[state])
    return action

def Q_learning(s, a, r, s_next, Q, eta, gamma, is_terminated):
    s = digitize_state(s)
    s_next = digitize_state(s_next)
    if is_terminated:
        Q[s, a] = Q[s, a] + eta * (r- Q[s, a])
    else:
        Q[s, a] = Q[s, a] + eta * (r + gamma * np.nanmax(Q[s_next, :]) - Q[s, a])
    return Q

def run_once(env, s_a_history,initial_observation, Q, epsilon, eta, gamma, pi, show=False):
    observation = initial_observation
    s = observation
    a = get_action(s, Q, epsilon, pi)
    a_next = a
    count = 0;
    while True:
        is_terminated = False
        if show:
            time.sleep(0.01)
            img = env.render()
            display_image(img)
        a = a_next
        s_a_history[-1][1] = a
        s_next, reward, terminated, truncated, info = env.step(a)
        s_a_history.append([s_next, np.nan])
        if count > 500:
            r = 1
            a_next = np.nan
            is_terminated = True
        elif count < 500 and terminated:
            r = -1
            a_next = np.nan
            is_terminated = True
        else:
            r = 0
            a_next = get_action(s_next, Q, epsilon, pi)
        Q = Q_learning(s, a, r, s_next, Q, eta, gamma, is_terminated)
        if is_terminated:
            if show:
                time.sleep(0.01)
                img = env.render()
                display_image(img)
            break
        else:
            s = s_next
        count += 1
    return [s_a_history, Q]

In [8]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
eta = 0.1
gamma = 0.9
epsilon = 0.5
Q = np.random.rand(NUM_DIZITIZED**4,2)
v = np.nanmax(Q, axis=1)
is_continue = True
episode = 1
env.reset()
while is_continue:
    observation, s_a_history, pi = reset(env)
    #print('episode: ' + str(episode))
    epsilon = epsilon / 2
    [s_a_history, Q] = run_once(env, s_a_history, observation, Q, epsilon, eta, gamma, pi,show=False)
    new_v = np.nanmax(Q, axis=1)
    #print(np.sum(np.abs(new_v - v)))
    v = new_v
    episode = episode + 1
    if episode > 1000:
        break
print(np.sum(np.abs(new_v - v)))
env.close()

0.0


In [29]:
env = gym.make('CartPole-v1', render_mode='rgb_array')
env.reset()
observation, s_a_history, pi = reset(env)
#run_once(env, s_a_history, observation, Q, 0, eta, gamma, pi,show=True)
env.observation_space.shape

(4,)